<h1 align='center'><span class="header-section-number"> </span>Diferenciación Automática con JAX <br/>Implementación de la función XOR</h1>

<h2>1.  Introducción </h2>

Con su versión actualizada de [Autograd](https://github.com/hips/autograd), [JAX](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html) puede diferenciar automáticamente el código nativo de Python y NumPy. Puede derivarse través de un gran subconjunto de características de Python, incluidos bucles, ifs, recursión y clousures, e incluso puede tomar derivadas de derivadas de derivadas. Admite la diferenciación tanto en modo inverso como en modo directo, y los dos pueden componerse arbitrariamente en cualquier orden.

Lo nuevo es que JAX usa [XLA](https://www.tensorflow.org/xla) para compilar y ejecutar su código NumPy en aceleradores, como GPU y TPU. La compilación ocurre de forma predeterminada, con las llamadas de la biblioteca compiladas y ejecutadas justo a tiempo. Pero JAX incluso le permite compilar justo a tiempo sus propias funciones de Python en núcleos optimizados para XLA utilizando una API de una función. La compilación y la diferenciación automática se pueden componer de forma arbitraria, por lo que puede expresar algoritmos sofisticados y obtener el máximo rendimiento sin tener que abandonar Python.


In [36]:
# !pip install --upgrade jax jaxlib 
from __future__ import print_function
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
key = random.PRNGKey(0)
# Current convention is to import original numpy as "onp"
import numpy as onp
import itertools

import random
import jax


<h2> 2. Función XOR </h2>

En este documento implementamos una red neuronal que calcula la función XOR. Esta es una función muy famosa en la historia de las redes neuronales artificiales, dado que fué la causante del primer invierno de estas.

La función lógica XOR es definida por

$$
\begin{aligned}
f(0,0) &= 1\\
f(0,1) &= 0\\
f(1,0) &=0\\
f(0,0) &=1
\end{aligned}
$$

Usaremos una red neuronal con una sola capa oculta con 3 neuronas y una no linealidad tangente hiperbólica, entrenada con la función de pérdida *entropía cruzada*, optimizando través del descenso de gradiente estocástico. Implementemos este modelo y la función de pérdida. Tenga en cuenta que el código es exactamente como lo escribiría en numpy estándar.
<h2> 3. Funciones Requeridas </h2>

In [37]:
# define the output activation
def sigmoid(x):
# more stable 0.5*(np.tanh(x/2)+1)
    return 1.0/(1+np.exp(-x))

# define the net
def net(params, x):
    w1, b1, w2, b2 = params
    hidden = np.tanh(np.dot(w1,x) + b1)
    return (sigmoid(np.dot(w2,hidden) + b2))

# cross entropy loss function
def loss(params, x,y):
    out = net(params,x)
    cross_entropy =  -y * np.log(out) - (1-y)*np.log(1-out) # esta es -log likelihood
    return cross_entropy

# Utility function for testing whether the net produces the correct
# output for all possible inputs
def test_all_inputs(inputs, params):
    predictions = [int(net(params, inp) > 0.5) for inp in inputs]
    for inp, out in zip(inputs, predictions):
        print(inp, '->', out)
    return (predictions == [onp.bitwise_xor(*inp) for inp in inputs])

Hay algunos lugares donde queremos usar numpy estándar en lugar de jax.numpy. Uno de esos lugares es con la inicialización de parámetros. Nos gustaría inicializar nuestros parámetros al azar antes de entrenar nuestra red, que no es una operación para la que necesitamos derivados o compilación. JAX usa su propia biblioteca jax.random en lugar de numpy.random que proporciona un mejor soporte para la reproducibilidad (siembra) a través de diferentes transformaciones. Dado que no necesitamos transformar la inicialización de los parámetros de ninguna manera, es más simple usar numpy.random estándar en lugar de jax.random aquí.

<h2> 4. jax.grad </h2>

La primera transformación que usaremos es *jax.grad*. *jax.grad* toma una función y devuelve una nueva función que calcula el gradiente de la función original. Por defecto, el gradiente se toma con respecto al primer argumento; esto se puede controlar mediante el argumento argnums de jax.grad. Para usar el gradiente descen diente, queremos poder calcular el gradiente de nuestra función de pérdida con respecto a los parámetros de nuestra red neuronal. 

In [39]:
loss_grad = grad(loss)

# Stochastic gradient descent
# Learning rate
learning_rate = 1.0
# all possible inputs 
inputs  = onp.array([[0,0],[0,1],[1,0],[1,1]])
targets = onp.array([0,1,1,0])
ide     =  onp.array([0,1,2,3])
# Initialize parameters randomly
params = initial_params()

In [40]:
params

[array([[ 0.25280359, -1.05972091],
        [-0.86363086, -0.57583006],
        [-0.06159423,  0.98018524]]),
 array([1.37862389, 0.94909999, 1.6194938 ]),
 array([-0.14456356,  0.42562094, -0.34343166]),
 -0.6278622829106619]

In [41]:
for n in itertools.count():
    # grab a single random input
    ix = ide[onp.random.choice(ide.shape[0])]
    # input
    x  = inputs[ix]
    # output
    y = targets[ix]
    # get the gradient  of the loss for this input/output losss
    grads = loss_grad(params,x,y)
    # update parameters via gradient descent
    params = [param - learning_rate * grad 
              for param, grad in zip(params,grads) ]
    # Every 100 iterations, check whether we've solve XOR
    if not n %100:
        print('Iteration{}'.format(n))
        if test_all_inputs(inputs, params):
            break
    

Iteration0
[0 0] -> 0
[0 1] -> 0
[1 0] -> 0
[1 1] -> 0
Iteration100
[0 0] -> 0
[0 1] -> 1
[1 0] -> 1
[1 1] -> 1
Iteration200
[0 0] -> 0
[0 1] -> 1
[1 0] -> 1
[1 1] -> 0


In [42]:
params

[DeviceArray([[ 3.7044861, -2.387884 ],
              [-2.489525 , -3.1954105],
              [-2.625351 ,  3.7185783]], dtype=float32),
 DeviceArray([0.79461575, 0.5428428 , 1.2141498 ], dtype=float32),
 DeviceArray([-3.5103402, -2.874178 , -3.5018072], dtype=float32),
 DeviceArray(0.32925892, dtype=float32)]

In [43]:
params

[DeviceArray([[ 3.7044861, -2.387884 ],
              [-2.489525 , -3.1954105],
              [-2.625351 ,  3.7185783]], dtype=float32),
 DeviceArray([0.79461575, 0.5428428 , 1.2141498 ], dtype=float32),
 DeviceArray([-3.5103402, -2.874178 , -3.5018072], dtype=float32),
 DeviceArray(0.32925892, dtype=float32)]

<h2> 5. jax.jit </h2>

Si bien el código numpy cuidadosamente escrito puede ser razonablemente eficaz, para el aprendizaje automático moderno queremos que nuestro código se ejecute lo más rápido posible. Esto a menudo implica ejecutar nuestro código en diferentes "aceleradores" como GPU o TPU. *JAX* proporciona un compilador *JIT* (justo a tiempo) que toma una función estándar de *Python/numpy* y la compila para ejecutarse eficientemente en un acelerador. Compilar una función también evita la sobrecarga del intérprete de Python, lo que ayuda tanto si está utilizando un acelerador como si no. En total, *jax.jit* puede acelerar drásticamente su código esencialmente sin sobrecarga de codificación; solo tiene que pedirle a JAX que compile la función por usted. Incluso nuestra pequeña red neuronal puede ver una aceleración bastante dramática al usar *jax.jit*:

In [44]:
# Time the original gradient function
%timeit loss_grad(params, x, y)
loss_grad = jax.jit(jax.grad(loss))
# Run once to trigger JIT compilation
loss_grad(params, x, y)
%timeit loss_grad(params, x, y)

16.8 ms ± 755 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
641 µs ± 124 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Let us run again the loop

In [46]:
for n in itertools.count():
    # grab a single random input
    ix = ide[onp.random.choice(ide.shape[0])]
    # input
    x  = inputs[ix]
    # output
    y = targets[ix]
    # get the gradient  of the loss for this input/output losss
    grads = loss_grad(params,x,y)
    # update parameters via gradient descent
    params = [param - learning_rate * grad 
              for param, grad in zip(params,grads) ]
    # Every 100 iterations, check whether we've solve XOR
    if not n %100:
        print('Iteration{}'.format(n))
        if test_all_inputs(inputs, params):
            break
    

Iteration0
[0 0] -> 0
[0 1] -> 1
[1 0] -> 1
[1 1] -> 0


<h2> 6.  jax.vmap </h2>

Hemos estado entrenando nuestra red neuronal en un solo ejemplo a la vez. Este es el "verdadero" descenso de gradiente estocástico; en la práctica, cuando entrenamos modelos modernos de aprendizaje automático, realizamos un descenso de gradiente “minibatch” donde promediamos los gradientes de pérdida en un mini lote de ejemplos en cada paso del descenso de gradiente. 

*JAX* proporciona *jax.vmap*, que es una transformación que automáticamente "vectoriza" una función. Lo que esto significa es que le permite calcular la salida de una función en paralelo sobre algún eje de la entrada. Para nosotros, esto significa que podemos aplicar la transformación de la función *jax.vmap* e inmediatamente obtener una versión de nuestro gradiente de la función de pérdida que es susceptible de utilizar un minibatch de ejemplos.

*jax.vmap* toma argumentos adicionales:

- *in_axes* es una tupla o número entero que le dice a *JAX* sobre qué ejes deben paralelizarse los argumentos de la función. La tupla debe tener la misma longitud que el número de argumentos de la función que se está vectorizando, o debe ser un número entero cuando solo hay un argumento. En nuestro ejemplo, usaremos *(None, 0, 0)*, que significa "no paralelizar sobre el primer argumento (parámetros), y paralelizar sobre la primera dimensión (cero) del segundo y tercer argumento (x e y) ".
- *out_axes* es análogo a in_axes, excepto que especifica qué ejes de la salida de la función se deben paralelizar. En nuestro caso, usaremos 0, que significa paralelizar sobre la primera dimensión (cero) de la única salida de la función (los gradientes de pérdida).

Tenga en cuenta que tendremos que cambiar un poco el código de entrenamiento: necesitamos obtener un lote de datos en lugar de un solo ejemplo a la vez, y debemos promediar los gradientes sobre el lote antes de aplicarlos para actualizar los parámetros.

In [47]:
loss_grad = jax.jit(jax.vmap(jax.grad(loss), in_axes=(None, 0, 0), out_axes=0))

params = initial_params()

batch_size = 100

for n in itertools.count():
    # Generate a batch of inputs
    x = inputs[onp.random.choice(inputs.shape[0], size=batch_size)]
    y = onp.bitwise_xor(x[:, 0], x[:, 1])
    # The call to loss_grad remains the same!
    grads = loss_grad(params, x, y)
    # Note that we now need to average gradients over the batch
    params = [param - learning_rate * np.mean(grad, axis=0)
              for param, grad in zip(params, grads)]
    if not n % 100:
        print('Iteration {}'.format(n))
        if test_all_inputs(inputs, params):
            break

Iteration 0
[0 0] -> 0
[0 1] -> 0
[1 0] -> 0
[1 1] -> 0
Iteration 100
[0 0] -> 0
[0 1] -> 1
[1 0] -> 1
[1 1] -> 0


In [49]:
params

[DeviceArray([[-2.7300384 ,  2.8734598 ],
              [ 0.0461734 ,  0.20690341],
              [-2.25555   ,  2.1526465 ]], dtype=float32),
 DeviceArray([ 1.4270214,  1.1485996, -1.0554334], dtype=float32),
 DeviceArray([-3.0328572,  1.0033779,  3.0124404], dtype=float32),
 DeviceArray(1.8351157, dtype=float32)]